# Importing required modules

In [43]:
from abc import ABC, abstractmethod
import random
import sys

# Building the environment model (simulator)

Abstract interface for any environment:

In [44]:
class BaseEnvironment(ABC):
    
    @abstractmethod
    # Override this method to return the set of states
    def get_states(self):
        pass
    
    @abstractmethod
    # Override this method to return the set of actions available in the state
    def get_actions(self, state):
        pass
    
    @abstractmethod
    def get_all_actions(self):
        pass
    
    @abstractmethod
    # Ovveride this method to implement p(next_state | current_state, action)
    def p(self, current_state, action, next_state):
        pass
    
    @abstractmethod
    # Override this method to implement r(current_state, action, next_state)
    def r(self, current_state, action, next_state):
        pass

<code>GridWorld</code> environment: a 4x4 grid where the agent can move up-down and left-right (no diagonal moves). Transitions are deterministic. Cell (3,3) is the target and it is the only one with a chance to stop. Rewards are -1 for every move and 10 when the target is reached.

In [45]:
class GridWorld(BaseEnvironment):
    
    UP = (-1,0)
    DOWN = (1,0)
    LEFT = (0,-1)
    RIGHT = (0,1)
    STAY = (0,0)
    
    label = {(-1,0):"UP", (1,0):"DOWN", (0,-1):"LEFT", (0,1):"RIGHT", (0,0):"STAY"}
    
    def __init__(self):
        self.states = list()
        for ri in range(4):
            for co in range(4):
                self.states.append((ri,co))
        self.actions = dict()
        self.actions[(0,0)] = [GridWorld.RIGHT, GridWorld.DOWN]
        self.actions[(0,1)] = [GridWorld.RIGHT, GridWorld.LEFT, GridWorld.DOWN]
        self.actions[(0,2)] = [GridWorld.RIGHT, GridWorld.LEFT, GridWorld.DOWN]
        self.actions[(0,3)] = [GridWorld.LEFT, GridWorld.DOWN]
        self.actions[(1,0)] = [GridWorld.RIGHT, GridWorld.DOWN, GridWorld.UP]
        self.actions[(1,1)] = [GridWorld.RIGHT, GridWorld.LEFT, GridWorld.DOWN, GridWorld.UP]
        self.actions[(1,2)] = [GridWorld.RIGHT, GridWorld.LEFT, GridWorld.DOWN, GridWorld.UP]
        self.actions[(1,3)] = [GridWorld.LEFT, GridWorld.DOWN, GridWorld.UP]
        self.actions[(2,0)] = [GridWorld.RIGHT, GridWorld.DOWN, GridWorld.UP]
        self.actions[(2,1)] = [GridWorld.RIGHT, GridWorld.LEFT, GridWorld.DOWN, GridWorld.UP]
        self.actions[(2,2)] = [GridWorld.RIGHT, GridWorld.LEFT, GridWorld.DOWN, GridWorld.UP]
        self.actions[(2,3)] = [GridWorld.LEFT, GridWorld.DOWN, GridWorld.UP]
        self.actions[(3,0)] = [GridWorld.RIGHT, GridWorld.UP]
        self.actions[(3,1)] = [GridWorld.RIGHT, GridWorld.LEFT, GridWorld.UP]
        self.actions[(3,2)] = [GridWorld.RIGHT, GridWorld.LEFT, GridWorld.UP]
        self.actions[(3,3)] = [GridWorld.STAY, GridWorld.LEFT, GridWorld.UP]
        self.all_actions = [GridWorld.STAY, GridWorld.RIGHT, GridWorld.LEFT, GridWorld.DOWN, GridWorld.UP]
    
    def get_states(self):
        return self.states
        
    def get_actions(self, state):
        return self.actions[state]
    
    def get_all_actions(self, state):
        return all_actions
    
    def transition(self, current_state, action):
        return (current_state[0] + action[0], current_state[1] + action[1]) 
    
    def p(self, current_state, action, next_state):
        # If the action is not executable in the state, whatever the next state is, the probability is 0
        if action not in self.get_actions(current_state):
            return 0
        # Compute the new state based on the action
        new_state = self.transition(current_state, action)
        # If the new state coincides with the next_state the probability is 1 
        # otherwise it is 0 (deterministic transitions)
        if new_state == next_state:
            return 1
        else:
            return 0
    
    def r(self, current_state, action, next_state):
        # If it is possible to reach the next state from the current one...
        if self.p(current_state, action, next_state) == 1:
            if (next_state != (3,3)):
                # ...the reward is -1 for every next state...
                return -1
            else:
                # ...except the goal 
                return 10
        # ... otherwise the reward is zero (just for correctness, but the robot will not go there)
        else:
            return 0

# Computing policies

Base class for all policies. The method <code>apply</code> enforces the policy, the method <code>update</code> changes the policy.

In [46]:
class BasePolicy(ABC):
    
    @abstractmethod
    # Ovveride this method to implement policy application
    # Returns the action given the state
    def apply(self, state):
        pass
    
    @abstractmethod
    # Ovveride this method to implement policy update
    # Sets the action to be returned for the given state
    def update(self, state, action):
        pass

A concrete policy to initialize policy-search algorithms. Initially, some action among those available for the state is chosen uniformly at random.

In [47]:
class CustomPolicy(BasePolicy):
    
    def __init__(self, environment):
        self.environment = environment
        self.state_action_table = dict()
        for state in environment.get_states():
            # Initially, no action is specified
            self.state_action_table[state] = None
        
    def apply(self, state):
        if self.state_action_table[state] == None:
            # When no action is specified, choose uniformly at random among available actions
            actions = self.environment.get_actions(state)
            return random.choice(actions)
        # If an action is available for the state, return that action
        return self.state_action_table[state]
    
    def update(self, state, action):
        self.state_action_table[state] = action

# Policy iteration algorithm

In [48]:
class PolicyIteration:
    
    def __init__(self, environment, gamma, theta):
        self.environment = environment
        self.gamma = gamma
        self.theta = theta
        self.policy = CustomPolicy(environment)
        # The value of states at step k
        self.old_value = dict()
        # The value of states at step k+1
        self.new_value = dict()
        # Initialize the value of each state to 0
        for state in environment.get_states():
            self.old_value[state] = 0
            
    def copy_values(self, from_value, to_value):
        for state in self.environment.get_states():
            to_value[state] = from_value[state]
            
    def policy_evaluation(self):
        while (True):
            delta = 0
            for state in self.environment.get_states():
                # Compute the new value of the state
                action = self.policy.apply(state)
                self.new_value[state] = 0
                for next_state in self.environment.get_states():
                    self.new_value[state] += self.environment.p(state, action, next_state)*\
                    (self.environment.r(state, action, next_state) + self.gamma * self.old_value[next_state])
                delta = max(delta, abs(self.old_value[state] - self.new_value[state]))
            # Copy the new value of states into the old ones
            self.copy_values(self.new_value, self.old_value)
            if delta < self.theta:
                return
            
    def policy_improvement(self):
        policy_stable = True
        for state in self.environment.get_states():
            old_action = self.policy.apply(state)
            # Compute the action yielding the maximum return
            max_value_of_action = -1 *  sys.float_info.max
            best_action = old_action
            for action in self.environment.get_actions(state):
                value_of_action = 0
                for next_state in self.environment.get_states():
                    value_of_action += self.environment.p(state, action, next_state) *\
                    (self.environment.r(state, action, next_state) + self.gamma * self.old_value[next_state])
                if value_of_action > max_value_of_action:
                    best_action = action
                    max_value_of_action = value_of_action
            if old_action != best_action:
                self.policy.update(state, best_action)
                policy_stable = False
        return policy_stable
    
    def apply(self):
        while (True):
            self.policy_evaluation()
            if self.policy_improvement():
                return

# Experiments with policy iteration

In [49]:
grid_world = GridWorld()
policy_iteration = PolicyIteration(grid_world, 0.8, 1/100000)

In [50]:
policy_iteration.apply()

Printing the optimal state values (optimal value function $v^*$):

In [51]:
for ri in range(4):
    for co in range(4):
        print(policy_iteration.old_value[(ri,co)],end=' ')
    print()

13.022400000000003 17.528000000000002 23.160000000000004 30.200000000000003 
17.528000000000002 23.160000000000004 30.200000000000003 39.0 
23.160000000000004 30.200000000000003 39.0 50.0 
30.200000000000003 39.0 50.0 50.0 


Printing the optimal policy $\pi^*$:

In [52]:
for ri in range(4):
    for co in range(4):
        print(GridWorld.label[policy_iteration.policy.apply((ri,co))],end=' ')
    print()

RIGHT RIGHT RIGHT DOWN 
RIGHT RIGHT RIGHT DOWN 
RIGHT RIGHT RIGHT DOWN 
RIGHT RIGHT RIGHT STAY 


As a comparison, let us check what happens with a single step of policy evaluation with a random policy:

In [53]:
policy_iteration = PolicyIteration(grid_world, 0.8, 1/100000)
policy_iteration.policy_evaluation()

In [54]:
for ri in range(4):
    for co in range(4):
        print(policy_iteration.old_value[(ri,co)],end=' ')
    print()

-4.999999999999998 -4.999999908292665 -4.999999999999998 -4.999999908292665 
-4.999999908292665 -4.999999999999998 -4.999999908292665 -4.999999999999998 
-4.999999999999998 -4.999999908292665 -4.999999999999998 -4.999999908292665 
-4.999999908292665 -4.999999999999998 -4.999999908292665 -4.999999999999998 


Now let us check a single policy improvement step:

In [55]:
result = policy_iteration.policy_improvement()
for ri in range(4):
    for co in range(4):
        print(GridWorld.label[policy_iteration.policy.apply((ri,co))],end=' ')
    print()

RIGHT LEFT DOWN LEFT 
RIGHT UP RIGHT UP 
UP RIGHT UP DOWN 
UP RIGHT RIGHT STAY 


As exxpected, the result is <code>False</code> since the initial policy was improved:

In [57]:
print(result)

False


Let us see another step to show the beginnings of convergence:

In [58]:
policy_iteration.policy_evaluation()
result = policy_iteration.policy_improvement()
for ri in range(4):
    for co in range(4):
        print(GridWorld.label[policy_iteration.policy.apply((ri,co))],end=' ')
    print()
print(result)

DOWN DOWN DOWN DOWN 
DOWN DOWN UP LEFT 
DOWN UP DOWN DOWN 
RIGHT RIGHT RIGHT STAY 
False


# Value Iteration algorithm

In [60]:
class ValueIteration:
    
    def __init__(self, environment, gamma, theta):
        self.environment = environment
        self.gamma = gamma
        self.theta = theta
        self.policy = CustomPolicy(environment)
        # The value of states at step k
        self.old_value = dict()
        # The value of states at step k+1
        self.new_value = dict()
        # Initialize the value of each state to 0
        for state in environment.get_states():
            self.old_value[state] = 0
            
    def copy_values(self, from_value, to_value):
        for state in self.environment.get_states():
            to_value[state] = from_value[state]
            
    def compute_values(self):
        while (True):
            delta = 0
            for state in self.environment.get_states():
                # Compute the new value of the state
                max_value = -1 * sys.float_info.max
                for action in self.environment.get_actions(state):
                    value = 0
                    for next_state in self.environment.get_states():
                        value += self.environment.p(state, action, next_state)*\
                        (self.environment.r(state, action, next_state) + self.gamma * self.old_value[next_state])
                    if value > max_value:
                        max_value = value
                self.new_value[state] = max_value
                delta = max(delta, abs(self.old_value[state] - self.new_value[state]))
            # Copy the new value of states into the old ones
            self.copy_values(self.new_value, self.old_value)
            if delta < self.theta:
                return
            
    def compute_policy(self):
        for state in self.environment.get_states():
            # Compute the action yielding the maximum return
            max_value_of_action = -1 * sys.float_info.max
            best_action = None
            for action in self.environment.get_actions(state):
                value_of_action = 0
                for next_state in self.environment.get_states():
                    value_of_action += self.environment.p(state, action, next_state) *\
                    (self.environment.r(state, action, next_state) + self.gamma * self.old_value[next_state])
                if value_of_action > max_value_of_action:
                    best_action = action
                    max_value_of_action = value_of_action
            self.policy.update(state, best_action)
    
    def apply(self):
        self.compute_values()
        self.compute_policy()

In [61]:
grid_world = GridWorld()
value_iteration = ValueIteration(grid_world, 0.8, 1/100000)

In [62]:
value_iteration.apply()

In [63]:
for ri in range(4):
    for co in range(4):
        print(GridWorld.label[value_iteration.policy.apply((ri,co))],end=' ')
    print()

RIGHT RIGHT RIGHT DOWN 
RIGHT RIGHT RIGHT DOWN 
RIGHT RIGHT RIGHT DOWN 
RIGHT RIGHT RIGHT STAY 
